# Prompting Large Language Models for Natural Language Inference

This notebook evaluates zero-shot and few-shot prompting strategies using LLaMA-3-8B-Instruct for the Natural Language Inference (NLI) task.

The goal is to compare prompting-based adaptation with supervised fine-tuning (RoBERTa-base) in terms of performance, stability, and computational cost.

## Task Definition

- Task: Natural Language Inference (NLI)
- Labels: entailment, contradiction, neutral
- Evaluation metric: Accuracy
- Output constraint: Model responses restricted to predefined label set

In [42]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
import numpy as np
import time
from tqdm import tqdm
import string

In [3]:
ds = load_dataset("stanfordnlp/snli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/413k [00:00<?, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [4]:
ds = ds.filter(lambda example: example['label'] != -1) #remove -1 labeled instances

labels = ds['train'].features['label'].names
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for i, l in enumerate(labels)}

val_ds = ds["validation"].select(range(100))

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [5]:
from huggingface_hub import login
login()

In [7]:
#load the model and its tokenizer
model_to_use = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_to_use)
model = AutoModelForCausalLM.from_pretrained(
    model_to_use,
    torch_dtype=torch.float16,
    device_map="auto"
)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Prompt Design

Two prompting strategies were evaluated:

1. Zero-shot prompting
2. Few-shot prompting (with 7 in-context examples)

Prompts were structured to explicitly restrict the output to one of the predefined labels.

In [116]:
#zero-shots prompt
def build_zero_shot_prompt(premise, hypothesis):
    messages = [
        {
            "role": "user",
            "content": f"""Determine the logical relationship between the premise and hypothesis.

Premise: {premise}
Hypothesis: {hypothesis}

IMPORTANT: Look carefully for contradictions - situations where the hypothesis is definitely false given the premise.

The relationship must be one of:
- entailment: the hypothesis is definitely true given the premise
- contradiction: the hypothesis is definitely false given the premise
- neutral: the hypothesis might or might not be true given the premise

Answer with only one word (entailment, contradiction, or neutral):"""
        }
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [117]:
#few-shots prompt
def build_few_shot_prompt(premise, hypothesis):
  messages = [
        {
            "role": "user",
            "content": f"""Classify the relationship as entailment, contradiction, or neutral.

IMPORTANT: Look carefully for contradictions - situations where the hypothesis is definitely false given the premise.

Premise: A soccer game with multiple males playing.
Hypothesis: Some men are playing a sport.
Answer: entailment

Premise: A child wearing a red coat is pointing into a lighted window.
Hypothesis: A child is wearing a jacket.
Answer: entailment

Premise: A man in a black shirt is looking at a bike in a workshop.
Hypothesis: A man is shopping for a car.
Answer: contradiction

Premise: A woman is outdoors.
Hypothesis: A woman is inside a building.
Answer: contradiction

Premise: The children are playing in the park.
Hypothesis: The children are sleeping in their beds.
Answer: contradiction

Premise: An older and younger man smiling.
Hypothesis: Two men are smiling and laughing at the cats playing on the floor.
Answer: neutral

Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition.
Answer: neutral

Premise: {premise}
Hypothesis: {hypothesis}
Answer with only one word:"""
        }
    ]
  return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [109]:
#prediction function
def predict_label(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len = inputs.input_ids.shape[1]

    #no track of the gradients
    with torch.no_grad():
        #defining output generation
        outputs = model.generate(
            **inputs, #ensuring all tensors are passed onto the model
            max_new_tokens=20, #making sure that it will generate labels as needed
            do_sample=False, #no random choices, only the highest probability token is chosen
            pad_token_id=tokenizer.eos_token_id
        )

    #convert token IDs to readable strings
    generated_tokens = outputs[0, input_len:]  #skip prompt tokens from output
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip().lower() #tokenize, remove punctuation and lower uppercase text

    #print for ensuring that the generated text is the respective label
    print(f"Raw output: '{text}'")

    #parsing the text
    text = text.split()[0] if text.split() else ""  #take first word only

    #encode the generated label as in the dataset
    if "entail" in text:
        return 0
    elif "neutral" in text:
        return 1
    elif "contra" in text:
        return 2
    else:
        return -1


In [110]:
#evaluation function
def evaluate(prompt_type, dataset):
    preds, refs = [], [] #store predictions and references

    start = time.time() #measure runtime
    #loop through the dataset and get predictions
    for ex in tqdm(dataset):
        prompt = prompt_type(ex["premise"], ex["hypothesis"]) #generate the prompt
        pred = predict_label(prompt) #get the prediction
        preds.append(pred) #add prediction to the list
        refs.append(ex["label"]) #add reference label to the list
    runtime = time.time() - start

    #calculate accuracy
    correct = sum(p == y for p, y in zip(preds, refs) if p != -1)
    total = sum(p != -1 for p in preds)

    accuracy = correct / total
    return accuracy, runtime, preds #return accuracy, runtime and predictions


In [118]:
#zero-shot runtime and accuracy
zero_acc, zero_time, zero_preds = evaluate(
    build_zero_shot_prompt,
    val_ds
)

print("Zero-shot accuracy:", zero_acc)
print("Zero-shot runtime (s):", zero_time)


  1%|          | 1/100 [00:03<05:52,  3.56s/it]

Raw output: 'entailment'


  2%|▏         | 2/100 [00:06<05:33,  3.40s/it]

Raw output: 'entailment'


  3%|▎         | 3/100 [00:08<04:15,  2.63s/it]

...

Raw output: 'entailment'


 99%|█████████▉| 99/100 [04:29<00:03,  3.25s/it]

Raw output: 'entailment'


100%|██████████| 100/100 [04:33<00:00,  2.73s/it]

Raw output: 'contradiction'
Zero-shot accuracy: 0.55
Zero-shot runtime (s): 273.16975712776184


In [119]:
#few-shot runtime and accuracy
few_acc, few_time, few_preds = evaluate(
    build_few_shot_prompt,
    val_ds
)

print("Few-shot accuracy:", few_acc)
print("Few-shot runtime (s):", few_time)


  1%|          | 1/100 [00:03<05:46,  3.50s/it]

Raw output: 'entailment'


  2%|▏         | 2/100 [00:07<05:49,  3.56s/it]

Raw output: 'entailment'


  3%|▎         | 3/100 [00:10<05:46,  3.58s/it]

...

Raw output: 'entailment'


 99%|█████████▉| 99/100 [05:15<00:03,  3.46s/it]

Raw output: 'entailment'


100%|██████████| 100/100 [05:18<00:00,  3.19s/it]

Raw output: 'contradiction'
Few-shot accuracy: 0.52
Few-shot runtime (s): 318.61034321784973


## Prompting discussion, decisions and instantiated prompts
-----------------------------------------------------------

Designing effective prompts for the Natural Language Inference task proved challenging, particularly when attempting to constrain the model to produce only a single label as output.

Zero-shot prompting
-------------------

The initial zero-shot prompt was relatively long and descriptive:


'''

You are performing Natural Language Inference task.

Given a Premise and a Hypothesis, classify their relationship as:
- entailment
- contradiction
- neutral

Definitions:
- entailment: the hypothesis must be true if the premise is true
- contradiction: the hypothesis cannot be true if the premise is true
- neutral: the hypothesis may or may not be true

Premise: {premise}
Hypothesis: {hypothesis}

Answer with one word only.

'''

However, with this prompt the model frequently produced long explanations or multiple labels instead of a single class. After experimenting with different prompt formulations, I adapted the prompt to better match the instruction format recommended for LLaMA models, based on the official LLaMA documentation (https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/). So, the prompt was adjusted as following:


'''

"role": "user",

"content": f"""Determine the logical relationship between the premise and hypothesis.

Premise: {premise}

Hypothesis: {hypothesis}

The relationship must be one of:
- entailment: the hypothesis is definitely true given the premise
- contradiction: the hypothesis is definitely false given the premise  
- neutral: the hypothesis might or might not be true given the premise

Answer with only one word (entailment, contradiction, or neutral):

'''


This version successfully constrained the model output to a single label. Using this prompt, the zero-shot model achieved an accuracy of 47% on the first 100 validation examples, with a runtime of approximately 4 minutes, which is substantially faster than fine-tuning but with much lower accuracy.

Few-shot prompting
------------------

A similar issue occurred when applying few-shot prompting with a generic prompt structure of the form:


'''

You are performing Natural Language Inference task.

Given a Premise and a Hypothesis, classify their relationship as:
- entailment
- contradiction
- neutral

{few_shot_examples}

Now classify the following:

Premise: {premise}
Hypothesis: {hypothesis}

Answer with one word only.

'''

Therefore, the few-shot prompt was also reformulated to follow the LLaMA instruction style:


'''

"role": "user",

"content": f"""Classify the relationship as entailment, contradiction, or neutral.

Example 1:
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition.
Answer: neutral

Example 2:
Premise: Children smiling and waving at camera
Hypothesis: There are children present
Answer: entailment

Example 3:
Premise: A boy is jumping on skateboard in the middle of a red bridge.
Hypothesis: The boy skates down the sidewalk.
Answer: contradiction

Premise: {premise}

Hypothesis: {hypothesis}

Answer with only one word (entailment, contradiction, or neutral):

'''

This approach resulted in slightly lower accuracy than the zero-shot version. Since the examples above were taken directly from the dataset, I experimented with manually created examples designed to be clearer and more general:

So the examples created initially were:

'''

Example 1:
Premise: A man is playing guitar.
Hypothesis: A man is playing a musical instrument.
Answer: entailment

Example 2:
Premise: A woman is indoors.
Hypothesis: A woman is outside.
Answer: contradiction

Example 3:
Premise: A dog is running in the park.
Hypothesis: The dog is brown.
Answer: neutral

'''

This modification led to a minor improvement. After inspecting the predicted labels, the model appeared to be biased toward the entailment class, while contradictions were rarely predicted.
To solve this issue, I added the following explicit instruction to both the zero-shot and few-shot prompts:

Instruction:

'''

IMPORTANT: Look carefully for contradictions - situations where the hypothesis is definitely false given the premise.

'''

In addition, the few-shot prompt was expanded to include more contradiction examples (three contradictions, two entailments, and two neutral examples). With these changes, accuracy improved to 55% for zero-shot prompting and 52% for few-shot prompting, while runtime remained low at approximately 5 minutes per evaluation.

Interestingly, few-shot prompting did not outperform zero-shot prompting, suggesting that the additional examples may have maintained bias or failed to generalize well to the SNLI dataset.


Some interesting examples of the instantiated prompts above are:

1. Example 3 in the validation set:

Premise: 'Two women are embracing while holding to go packages.'
Hypothesis: 'The men are fighting outside a deli.'
True Label: Contradiction
Zero-Shot label: Neutral
Few-Shots label: Contradiction

Few-Shots model predicts accurately the label, while zero-shot cannot.

2. Example 5 in validation set:

Premise: 'Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink.'
Hypothesis: Two kids at a ballgame wash their hands.
True Label: Neutral
Zero-Shot label: Neutral
Few-Shots label: Entailment

Here, zero-shot predicts correctly, while few-shots doesn't.

3. Example 30 in validation set:

Premise: 'Families waiting in line at an amusement park for their turn to ride.'
Hypothesis: 'People are waiting to see a movie.'
True Label: Contradiction
Zero-Shot label: Contradiction
Few-Shots label: Neutral

Again, zero-shot seems to be predicting correctly the contradiction, while few-shots confuses it for neutral.

Overall, these examples highlight the difficulty of controlling LLM behavior through prompting alone. The observed bias toward entailment likely contributes to errors, particularly for contradiction cases. Due to time constraints, further prompt engineering was not explored.

## Comparison of fine-tuning with prompting in 100 samples validation subset

Accuracy of fine-tuned model: 96%

Accuracy of zero-shot model: 55%

Accuracy of few-shots model: 52%


Runtime of fine-tuned model: 20s

Runtime of zero-shot model: 273s

Runtime of few-shots model: 318s


The fine-tuned RoBERTa model outperforms both the zero-shot and few-shot prompting approaches in terms of accuracy and runtime when all models are evaluated on the same 100 examples from the SNLI validation set. This result is expected, as fine-tuning explicitly adapts the model parameters to the NLI task and the label distribution of the dataset.
However, this process has a high computational cost. Fine-tuning RoBERTa (while freezing the embeddings and half of the encoder layers and training only the remaining encoder layers and the classification head) required approximately 1 hour of training time on a GPU in Google Colab. In contrast, the prompting approaches required no training time and no training data. But, although prompting is more flexible and data-efficient, its performance in this task was close to chance level, indicating that the LLM struggled to consistently infer the correct NLI labels, even when few-shot examples were provided. This suggests that, for structured classification tasks such as NLI, prompting without task-specific training may be insufficient.
